# QUESTON 1

In [7]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [8]:
data=load_iris()
X=data.data
y=data.target
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [47]:
D={}
diff_class=np.unique(y)
for j in diff_class :
    D[j]=[];
for i,j in zip(X,y):
    D[j].append(i)
for j in diff_class :
    D[j]=np.asarray(D[j])

In [48]:
whole_mean=np.mean(X,axis=0)
each_class_mean=[]
for key in D:
    each_class_mean.append(np.mean(D[key],axis=0))
each_class_mean=np.asarray(each_class_mean)

In [58]:
# within class scatter matrix 
Sw=np.zeros((X.shape[1],X.shape[1]))
for c in diff_class :
    Sw+=np.matmul((D[c]-each_class_mean[c]).T,(D[c]-each_class_mean[c]))

In [59]:
# between class scatter matrix
Sb=np.zeros((X.shape[1],X.shape[1]))
Sb+=len(D[c])*np.matmul((whole_mean-each_class_mean).T,(whole_mean-each_class_mean))

In [61]:
scatter_matrix=np.matmul(np.linalg.inv(Sw),Sb)

In [67]:
eig_val,eig_vec=np.linalg.eig(scatter_matrix)
sorted_idxs=np.argsort(eig_val)       # getting sorted indices of eigen values
eig_val=np.sort(eig_val)
j=1
for i in range(len(eig_val)-1,-1,-1) :
    print("percentage of explained variance for principle component {} = {}".format(j,eig_val[i]/np.sum(eig_val)))
    j+=1;

percentage of explained variance for principle component 1 = 0.9914724756595078
percentage of explained variance for principle component 2 = 0.008527524340492412
percentage of explained variance for principle component 3 = 5.098172111923971e-17
percentage of explained variance for principle component 4 = -2.687844842177008e-16


In [68]:
sum=0
j=0;
for i in range(len(eig_val)-1,-1,-1) :
    sum+=(eig_val[i]/np.sum(eig_val))
    j+=1 
    if(sum>0.99): break                          # if current variance information exceeded 98% percentage we can stop
X_new=np.matmul(X,eig_vec[sorted_idxs[:j]].T)    # transformed data
X_new.shape

(150, 1)

# QUESTION 2 

In [97]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [98]:
data=pd.read_csv("wine.csv")
data.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [99]:
X=data.iloc[:,:-2]
y=data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [100]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)

In [101]:
lda=LinearDiscriminantAnalysis()
lda.fit(X_train,y_train)
X_red_train=lda.transform(X_train)

In [102]:
clf=LogisticRegression()
clf.fit(X_red_train,y_train)
clf.score(X_red_train,y_train)

0.9924812030075187

In [103]:
# model training completed
# Now testing dataset 
lda1=LinearDiscriminantAnalysis(n_components=X_red_train.shape[1])
X_red_test=lda1.fit_transform(X_test,y_test)
scaler.fit(X_red_test)
X_res_test=scaler.transform(X_red_test)
y_pred=clf.predict(X_red_test)

In [104]:
confusion_matrix(y_test,y_pred) # confusion matrix

array([[ 9,  0,  0],
       [ 0, 23,  1],
       [ 0,  0, 12]], dtype=int64)